In [ ]:
import os
import sys
file_dir = os.path.dirname(os.path.dirname(os.path.abspath("/home/rodr/code/mobility-baselines/STG-NCDE-main")))
print(file_dir)
sys.path.append(file_dir)

import torch
import numpy as np
import torch.nn as nn
import argparse
import configparser
import time

from model.BasicTrainer_cde import Trainer
from lib.TrainInits import init_seed
from lib.dataloader import get_dataloader_cde
from lib.TrainInits import print_model_parameters
import os
from os.path import join
from model.Make_model import make_model
from torch.utils.tensorboard import SummaryWriter
#*************************************************************************#
Mode = 'train'
DEBUG = 'False'
DATASET = 'PEMSD4'      #PEMSD4 or PEMSD8
MODEL = 'GCDE'

#get configuration
config_file = 'model/{}_{}.conf'.format(DATASET, MODEL)
#print('Read configuration file: %s' % (config_file))
config = configparser.ConfigParser()
config.read(config_file)

from lib.metrics import MAE_torch
def masked_mae_loss(scaler, mask_value):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        mae = MAE_torch(pred=preds, true=labels, mask_value=mask_value)
        return mae
    return loss

# python Run_cde.py --dataset='PEMSD4' --model='GCDE' --model_type='type1' --embed_dim=10 --hid_dim=64 --hid_hid_dim=64 --num_layers=2 --lr_init=0.001 --weight_decay=1e-3 --epochs=200 --tensorboard --comment="" --device=0

#parser
args = argparse.ArgumentParser(description='arguments')
args.add_argument('--dataset', default=DATASET, type=str)
args.add_argument('--mode', default=Mode, type=str)
args.add_argument('--device', default=0, type=int, help='indices of GPUs')
args.add_argument('--debug', default=DEBUG, type=eval)
args.add_argument('--model', default=MODEL, type=str)
args.add_argument('--cuda', default=True, type=bool)
args.add_argument('--comment', default='', type=str)
args.add_argument("-f", type=str, default="")


#data
args.add_argument('--val_ratio', default=config['data']['val_ratio'], type=float)
args.add_argument('--test_ratio', default=config['data']['test_ratio'], type=float)
args.add_argument('--lag', default=config['data']['lag'], type=int)
args.add_argument('--horizon', default=config['data']['horizon'], type=int)
args.add_argument('--num_nodes', default=config['data']['num_nodes'], type=int)
args.add_argument('--tod', default=config['data']['tod'], type=eval)
args.add_argument('--normalizer', default=config['data']['normalizer'], type=str)
args.add_argument('--column_wise', default=config['data']['column_wise'], type=eval)
args.add_argument('--default_graph', default=config['data']['default_graph'], type=eval)
#model
args.add_argument('--model_type', default=config['model']['type'], type=str)
args.add_argument('--g_type', default=config['model']['g_type'], type=str)
args.add_argument('--input_dim', default=config['model']['input_dim'], type=int)
args.add_argument('--output_dim', default=config['model']['output_dim'], type=int)
args.add_argument('--embed_dim', default=config['model']['embed_dim'], type=int)
args.add_argument('--hid_dim', default=config['model']['hid_dim'], type=int)
args.add_argument('--hid_hid_dim', default=config['model']['hid_hid_dim'], type=int)
args.add_argument('--num_layers', default=config['model']['num_layers'], type=int)
args.add_argument('--cheb_k', default=config['model']['cheb_order'], type=int)
args.add_argument('--solver', default='rk4', type=str)

#train
args.add_argument('--loss_func', default=config['train']['loss_func'], type=str)
args.add_argument('--seed', default=config['train']['seed'], type=int)
args.add_argument('--batch_size', default=config['train']['batch_size'], type=int)
args.add_argument('--epochs', default=config['train']['epochs'], type=int)
args.add_argument('--lr_init', default=config['train']['lr_init'], type=float)
args.add_argument('--weight_decay', default=config['train']['weight_decay'], type=eval)
args.add_argument('--lr_decay', default=config['train']['lr_decay'], type=eval)
args.add_argument('--lr_decay_rate', default=config['train']['lr_decay_rate'], type=float)
args.add_argument('--lr_decay_step', default=config['train']['lr_decay_step'], type=str)
args.add_argument('--early_stop', default=config['train']['early_stop'], type=eval)
args.add_argument('--early_stop_patience', default=config['train']['early_stop_patience'], type=int)
args.add_argument('--grad_norm', default=config['train']['grad_norm'], type=eval)
args.add_argument('--max_grad_norm', default=config['train']['max_grad_norm'], type=int)
args.add_argument('--teacher_forcing', default=False, type=bool)
#args.add_argument('--tf_decay_steps', default=2000, type=int, help='teacher forcing decay steps')
args.add_argument('--real_value', default=config['train']['real_value'], type=eval, help = 'use real value for loss calculation')

args.add_argument('--missing_test', default=False, type=bool)
args.add_argument('--missing_rate', default=0.1, type=float)

#test
args.add_argument('--mae_thresh', default=config['test']['mae_thresh'], type=eval)
args.add_argument('--mape_thresh', default=config['test']['mape_thresh'], type=float)
args.add_argument('--model_path', default='', type=str)
#log
args.add_argument('--log_dir', default='../runs', type=str)
args.add_argument('--log_step', default=config['log']['log_step'], type=int)
args.add_argument('--plot', default=config['log']['plot'], type=eval)
args.add_argument('--tensorboard',action='store_true',help='tensorboard')

args = args.parse_args()
init_seed(args.seed)

GPU_NUM = args.device
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

print(args)

#config log path
save_name = time.strftime("%m-%d-%Hh%Mm")+args.comment+"_"+ args.dataset+"_"+ args.model+"_"+ args.model_type+"_"+"embed{"+str(args.embed_dim)+"}"+"hid{"+str(args.hid_dim)+"}"+"hidhid{"+str(args.hid_hid_dim)+"}"+"lyrs{"+str(args.num_layers)+"}"+"lr{"+str(args.lr_init)+"}"+"wd{"+str(args.weight_decay)+"}"
path = '../runs'

log_dir = join(path, args.dataset, save_name)
args.log_dir = log_dir
if (os.path.exists(args.log_dir)):
        print('has model save path')
else:
    os.makedirs(args.log_dir)

if args.tensorboard:
    w : SummaryWriter = SummaryWriter(args.log_dir)
else:
    w = None

#init model
if args.model_type=='type1':
    model, vector_field_f, vector_field_g = make_model(args)
elif args.model_type=='type1_temporal':
    model, vector_field_f = make_model(args)
elif args.model_type=='type1_spatial':
    model, vector_field_g = make_model(args)
else:
    raise ValueError("Check args.model_type")

model = model.to(args.device)

if args.model_type=='type1_temporal':
    vector_field_f = vector_field_f.to(args.device)
    vector_field_g = None
elif args.model_type=='type1_spatial':
    vector_field_f = None
    vector_field_g = vector_field_g.to(args.device)
else:
    vector_field_f = vector_field_f.to(args.device)
    vector_field_g = vector_field_g.to(args.device)

print(model)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    else:
        nn.init.uniform_(p)
print_model_parameters(model, only_num=False)

#load dataset
train_loader, val_loader, test_loader, scaler, times = get_dataloader_cde(args,
                                                               normalizer=args.normalizer,
                                                               tod=args.tod, dow=False,
                                                               weather=False, single=False)

#init loss function, optimizer
if args.loss_func == 'mask_mae':
    loss = masked_mae_loss(scaler, mask_value=0.0)
elif args.loss_func == 'mae':
    loss = torch.nn.L1Loss().to(args.device)
elif args.loss_func == 'mse':
    loss = torch.nn.MSELoss().to(args.device)
elif args.loss_func == 'huber_loss':
    loss = torch.nn.HuberLoss(delta=1.0).to(args.device)
else:
    raise ValueError

optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr_init,
                             weight_decay=args.weight_decay)

#learning rate decay
lr_scheduler = None
if args.lr_decay:
    print('Applying learning rate decay.')
    lr_decay_steps = [int(i) for i in list(args.lr_decay_step.split(','))]
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
                                                        milestones=lr_decay_steps,
                                                        gamma=args.lr_decay_rate)

#start training
trainer = Trainer(model, vector_field_f, vector_field_g, loss, optimizer, train_loader, val_loader, test_loader, scaler,
                  args, lr_scheduler, args.device, times,
                  w)
if args.mode == 'train':
    trainer.train()
elif args.mode == 'test':
    model.load_state_dict(torch.load('./pre-trained/{}.pth'.format(args.dataset)))
    print("Load saved model")
    trainer.test(model, trainer.args, test_loader, scaler, trainer.logger, times)
else:
    raise ValueError


/home/rodr/code
Namespace(batch_size=64, cheb_k=2, column_wise=False, comment='', cuda=True, dataset='PEMSD4', debug=False, default_graph=True, device=0, early_stop=True, early_stop_patience=15, embed_dim=10, epochs=100, f='/home/rodr/.local/share/jupyter/runtime/kernel-6038d5ec-3b1c-4b45-80bd-260bf458760e.json', g_type='agc', grad_norm=False, hid_dim=128, hid_hid_dim=128, horizon=12, input_dim=2, lag=12, log_dir='../runs', log_step=20, loss_func='mae', lr_decay=False, lr_decay_rate=0.3, lr_decay_step='5,20,40,70', lr_init=0.001, mae_thresh=None, mape_thresh=0.0, max_grad_norm=5, missing_rate=0.1, missing_test=False, mode='train', model='GCDE', model_path='', model_type='type1', normalizer='std', num_layers=3, num_nodes=307, output_dim=1, plot=False, real_value=True, seed=10, solver='rk4', teacher_forcing=False, tensorboard=False, test_ratio=0.2, tod=False, val_ratio=0.2, weight_decay=0.001)
NeuralGCDE(
  (func_f): FinalTanh_f(
    input_channels: 2, hidden_channels: 128, hidden_hidden

2022-02-10 17:23: Experiment log path in: ../runs/PEMSD4/02-10-17h23m_PEMSD4_GCDE_type1_embed{10}hid{128}hidhid{128}lyrs{3}lr{0.001}wd{0.001}
2022-02-10 17:23: Argument batch_size: 64
2022-02-10 17:23: Argument cheb_k: 2
2022-02-10 17:23: Argument column_wise: False
2022-02-10 17:23: Argument comment: ''
2022-02-10 17:23: Argument cuda: True
2022-02-10 17:23: Argument dataset: 'PEMSD4'
2022-02-10 17:23: Argument debug: False
2022-02-10 17:23: Argument default_graph: True
2022-02-10 17:23: Argument device: 0
2022-02-10 17:23: Argument early_stop: True
2022-02-10 17:23: Argument early_stop_patience: 15
2022-02-10 17:23: Argument embed_dim: 10
2022-02-10 17:23: Argument epochs: 100
2022-02-10 17:23: Argument f: '/home/rodr/.local/share/jupyter/runtime/kernel-6038d5ec-3b1c-4b45-80bd-260bf458760e.json'
2022-02-10 17:23: Argument g_type: 'agc'
2022-02-10 17:23: Argument grad_norm: False
2022-02-10 17:23: Argument hid_dim: 128
2022-02-10 17:23: Argument hid_hid_dim: 128
2022-02-10 17:23: Argu

Creat Log File in:  ../runs/PEMSD4/02-10-17h23m_PEMSD4_GCDE_type1_embed{10}hid{128}hidhid{128}lyrs{3}lr{0.001}wd{0.001}/run.log
*****************Model Parameter*****************
node_embeddings torch.Size([307, 10]) True
func_f.linear_in.weight torch.Size([128, 128]) True
func_f.linear_in.bias torch.Size([128]) True
func_f.linears.0.weight torch.Size([128, 128]) True
func_f.linears.0.bias torch.Size([128]) True
func_f.linears.1.weight torch.Size([128, 128]) True
func_f.linears.1.bias torch.Size([128]) True
func_f.linear_out.weight torch.Size([256, 128]) True
func_f.linear_out.bias torch.Size([256]) True
func_g.node_embeddings torch.Size([307, 10]) True
func_g.weights_pool torch.Size([10, 2, 128, 128]) True
func_g.bias_pool torch.Size([10, 128]) True
func_g.linear_in.weight torch.Size([128, 128]) True
func_g.linear_in.bias torch.Size([128]) True
func_g.linear_out.weight torch.Size([16384, 128]) True
func_g.linear_out.bias torch.Size([16384]) True
end_conv.weight torch.Size([12, 1, 1, 12

2022-02-10 17:23: Train Epoch 1: 0/158 Loss: 187.581009
2022-02-10 17:24: Train Epoch 1: 20/158 Loss: 57.625172
2022-02-10 17:25: Train Epoch 1: 40/158 Loss: 39.290855
2022-02-10 17:26: Train Epoch 1: 60/158 Loss: 32.908375
2022-02-10 17:28: Train Epoch 1: 80/158 Loss: 35.512909
2022-02-10 17:29: Train Epoch 1: 100/158 Loss: 40.473694
2022-02-10 17:30: Train Epoch 1: 120/158 Loss: 31.749498
2022-02-10 17:31: Train Epoch 1: 140/158 Loss: 28.171183
2022-02-10 17:32: **********Train Epoch 1: averaged Loss: 47.409154
2022-02-10 17:33: **********Val Epoch 1: average Loss: 31.252103
2022-02-10 17:33: *********************************Current best model saved!
2022-02-10 17:33: Train Epoch 2: 0/158 Loss: 32.205299
2022-02-10 17:35: Train Epoch 2: 20/158 Loss: 30.635521
2022-02-10 17:36: Train Epoch 2: 40/158 Loss: 26.449610
2022-02-10 17:37: Train Epoch 2: 60/158 Loss: 24.155886
2022-02-10 17:38: Train Epoch 2: 80/158 Loss: 27.650103
2022-02-10 17:40: Train Epoch 2: 100/158 Loss: 25.718758
202

2022-02-10 19:47: Train Epoch 13: 140/158 Loss: 20.809467
2022-02-10 19:48: **********Train Epoch 13: averaged Loss: 19.691564
2022-02-10 19:49: **********Val Epoch 13: average Loss: 21.213859
2022-02-10 19:49: Train Epoch 14: 0/158 Loss: 19.771690
2022-02-10 19:51: Train Epoch 14: 20/158 Loss: 19.351835
2022-02-10 19:52: Train Epoch 14: 40/158 Loss: 19.413378
2022-02-10 19:53: Train Epoch 14: 60/158 Loss: 20.714151
2022-02-10 19:54: Train Epoch 14: 80/158 Loss: 20.464701
2022-02-10 19:56: Train Epoch 14: 100/158 Loss: 20.458221
2022-02-10 19:57: Train Epoch 14: 120/158 Loss: 19.932886
2022-02-10 19:58: Train Epoch 14: 140/158 Loss: 18.826841
2022-02-10 19:59: **********Train Epoch 14: averaged Loss: 19.595515
2022-02-10 20:00: **********Val Epoch 14: average Loss: 20.435457
2022-02-10 20:00: *********************************Current best model saved!
2022-02-10 20:00: Train Epoch 15: 0/158 Loss: 20.843346
2022-02-10 20:02: Train Epoch 15: 20/158 Loss: 18.235737
2022-02-10 20:03: Train 

2022-02-10 22:14: Train Epoch 27: 0/158 Loss: 18.403982
2022-02-10 22:16: Train Epoch 27: 20/158 Loss: 17.235260
2022-02-10 22:17: Train Epoch 27: 40/158 Loss: 19.226721
2022-02-10 22:18: Train Epoch 27: 60/158 Loss: 21.107458
2022-02-10 22:19: Train Epoch 27: 80/158 Loss: 18.809050
2022-02-10 22:21: Train Epoch 27: 100/158 Loss: 23.186954
2022-02-10 22:22: Train Epoch 27: 120/158 Loss: 28.334248
2022-02-10 22:23: Train Epoch 27: 140/158 Loss: 27.366379
2022-02-10 22:24: **********Train Epoch 27: averaged Loss: 22.360649
2022-02-10 22:25: **********Val Epoch 27: average Loss: 29.309803
2022-02-10 22:25: Train Epoch 28: 0/158 Loss: 30.227592
2022-02-10 22:27: Train Epoch 28: 20/158 Loss: 21.539219
2022-02-10 22:28: Train Epoch 28: 40/158 Loss: 20.637924
2022-02-10 22:29: Train Epoch 28: 60/158 Loss: 19.880579
2022-02-10 22:30: Train Epoch 28: 80/158 Loss: 20.298031
2022-02-10 22:32: Train Epoch 28: 100/158 Loss: 17.868605
2022-02-10 22:33: Train Epoch 28: 120/158 Loss: 20.308828
2022-02

2022-02-11 00:53: Train Epoch 40: 120/158 Loss: 25.119024
2022-02-11 00:55: Train Epoch 40: 140/158 Loss: 28.762632
2022-02-11 00:56: **********Train Epoch 40: averaged Loss: 26.556906
2022-02-11 00:57: **********Val Epoch 40: average Loss: 29.552981
2022-02-11 00:57: Train Epoch 41: 0/158 Loss: 29.481506
2022-02-11 00:58: Train Epoch 41: 20/158 Loss: 34.286579
2022-02-11 00:59: Train Epoch 41: 40/158 Loss: 30.762569
2022-02-11 01:00: Train Epoch 41: 60/158 Loss: 52.150764
2022-02-11 01:02: Train Epoch 41: 80/158 Loss: 30.732565
2022-02-11 01:03: Train Epoch 41: 100/158 Loss: 26.971004
2022-02-11 01:04: Train Epoch 41: 120/158 Loss: 29.567608
2022-02-11 01:06: Train Epoch 41: 140/158 Loss: 29.813307
2022-02-11 01:07: **********Train Epoch 41: averaged Loss: 32.829339
2022-02-11 01:08: **********Val Epoch 41: average Loss: 28.849851
2022-02-11 01:08: Train Epoch 42: 0/158 Loss: 28.395657
2022-02-11 01:09: Train Epoch 42: 20/158 Loss: 25.838446
2022-02-11 01:10: Train Epoch 42: 40/158 Lo